# General code for road network

In [ ]:
import geopandas as gpd
import folium 
import json

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Functions: Getting RWS network

def DutchRDtoWGS84(rdX, rdY):
    """ Convert DutchRD to WGS84
    """
    RD_MINIMUM_X = 11000
    RD_MAXIMUM_X = 280000
    RD_MINIMUM_Y = 300000
    RD_MAXIMUM_Y = 630000
    if (rdX < RD_MINIMUM_X or rdX > RD_MAXIMUM_X
        or rdY < RD_MINIMUM_Y or rdY > RD_MAXIMUM_Y):
        resultNorth = -1
        resultEast = -1
        return resultNorth, resultEast
    # else
    dX = (rdX - 155000.0) / 100000.0
    dY = (rdY - 463000.0) / 100000.0
    k = [[3600 * 52.15517440, 3235.65389, -0.24750, -0.06550, 0.0],
        [-0.00738   ,   -0.00012,  0.0    ,  0.0    , 0.0],
        [-32.58297   ,   -0.84978, -0.01709, -0.00039, 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [0.00530   ,    0.00033,  0.0    ,  0.0    , 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0]]
    l = [[3600 * 5.38720621,    0.01199,  0.00022,  0.0    , 0.0],
        [5260.52916   ,  105.94684,  2.45656,  0.05594, 0.00128],
        [-0.00022   ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [-0.81885   ,   -0.05607, -0.00256,  0.0    , 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [0.00026   ,    0.0    ,  0.0    ,  0.0    , 0.0]]
    resultNorth = 0
    resultEast = 0
    powX = 1

    for p in range(6):
        powY = 1
        for q in range(5):
            resultNorth = resultNorth + k[p][q] * powX * powY / 3600.0
            resultEast = resultEast + l[p][q] * powX * powY / 3600.0
            powY = powY * dY
        powX = powX * dX
    return resultNorth, resultEast

def WGS84toDutchRD(wgs84East, wgs84North):
    # translated from Peter Knoppers's code

    # wgs84East: longtitude
    # wgs84North: latitude

    # Western boundary of the Dutch RD system. */
    WGS84_WEST_LIMIT = 3.2

    # Eastern boundary of the Dutch RD system. */
    WGS84_EAST_LIMIT = 7.3

    # Northern boundary of the Dutch RD system. */
    WGS84_SOUTH_LIMIT = 50.6

    # Southern boundary of the Dutch RD system. */
    WGS84_NORTH_LIMIT = 53.7

    if (wgs84North > WGS84_NORTH_LIMIT) or \
        (wgs84North < WGS84_SOUTH_LIMIT) or \
        (wgs84East < WGS84_WEST_LIMIT) or \
        (wgs84East > WGS84_EAST_LIMIT):
        resultX = -1
        resultY = -1
    else:
        r = [[155000.00, 190094.945,   -0.008, -32.391, 0.0],
            [-0.705, -11832.228,    0.0  ,   0.608, 0.0],
            [0.0  ,   -114.221,    0.0  ,   0.148, 0.0],
            [0.0  ,     -2.340,    0.0  ,   0.0  , 0.0],
            [0.0  ,      0.0  ,    0.0  ,   0.0  , 0.0]]
        s = [[463000.00 ,      0.433, 3638.893,   0.0  ,  0.092],
            [309056.544,     -0.032, -157.984,   0.0  , -0.054],
            [73.077,      0.0  ,   -6.439,   0.0  ,  0.0],
            [59.788,      0.0  ,    0.0  ,   0.0  ,  0.0],
            [0.0  ,      0.0  ,    0.0  ,   0.0  ,  0.0]]
        resultX = 0
        resultY = 0
        powNorth = 1
        dNorth = 0.36 * (wgs84North - 52.15517440)
        dEast = 0.36 * (wgs84East - 5.38720621)

        for p in range(5):
            powEast = 1
            for q in range(5):
                resultX = resultX + r[p][q] * powEast * powNorth
                resultY = resultY + s[p][q] * powEast * powNorth
                powEast = powEast * dEast
            powNorth = powNorth * dNorth
    return resultX, resultY

def calc_distance(line_wkt):
    line = ogr.CreateGeometryFromWkt(line_wkt)
    points = line.GetPoints()
    d = 0
    for p0, p1 in zip(points, points[1:]):
        d = d + geodesic(p0, p1).m
    return d

if __name__=="__main__":
    x, y = WGS84toDutchRD(4.33, 52.04)

In [ ]:
#Extract subnetwork
highway_shapefile = 'Snelheid_Wegvakken.shp'
network_temp = gpd.read_file(highway_shapefile)

# Method: Clustering

In [ ]:
from pyproj import Transformer
from matplotlib.colors import ListedColormap

In [ ]:
n = 50
inc = pd.read_csv('Incidents_clean.csv')

In [ ]:
customcmap = ListedColormap(["lightcoral", "firebrick", "darkred", "red", "coral", "orangered", "crimson", "sandybrown", "peru", "darkorange", "navajowhite", "moccasin", "orange", "darkgoldenrod", "gold", "darkkhaki", "olive", "yellow", "yellowgreen", "lawngreen", "darkseagreen", "palegreen", "green", "darkgreen", "springgreen", "aquamarine", "turquoise", "lightseagreen", "paleturquoise", "teal", "cyan", "cadetblue", "deepskyblue", "steelblue", "dodgerblue", "slategray", "midnightblue", "blue", "mediumslateblue", "mediumpurple", "blueviolet", "plum", "violet", "purple", "magenta", "deeppink", "pink", "mediumblue", "darkmagenta", "palevioletred"])

### 1. Clustering functions based on centroids

In [ ]:
def initiate_centroids(n, list_inc):
    u = list_inc.sample(n)
    return u

In [ ]:
def calc_err(xy1, xy2):
    return np.sqrt(np.sum((xy1 - xy2)**2))

In [ ]:
def centroid_assignation(list_inc, u):
    n_i = len(list_inc)
    centroid_near_inc = np.zeros(n_i)
    centroid_error = np.zeros(n_i)

    for i in range(n_i):
        e = np.zeros(n)
        for c in range(n):
            e[c] = calc_err(u.iloc[c, :], list_inc.iloc[i,:])

        centroid_near_inc[i] =  np.where(e==np.amin(e))[0].tolist()[0]
        centroid_error[i] = np.amin(e)

    return centroid_near_inc, centroid_error

In [ ]:
def clustering(list_inc, n=50, tol=1e-4):
    new_list_inc = list_inc.copy()
    iterations_error = []
    continue_yn = True
    iteration = 0
    u_s = initiate_centroids(n, list_inc)

    while(continue_yn):
        # Step 3 and 4 - Assign centroids and calculate error
        new_list_inc['centroid'], u_s_error = centroid_assignation(new_list_inc, u_s) 
        iterations_error.append(np.sum(u_s_error))
        
        # Step 5 - Update centroid position
        centroids = new_list_inc.groupby('centroid').agg('mean').reset_index(drop = True)

        # Step 6 - Restart the iteration
        if iteration>0:
            # Is the error less than a tolerance (1E-4)
            if iterations_error[iteration - 1] - iterations_error[iteration] <= tol:
                continue_yn = False
        iteration+=1
        print(iteration)

    new_list_inc['centroid'], u_s_error = centroid_assignation(new_list_inc, u_s)
    u_s = new_list_inc.groupby('centroid').agg('mean').reset_index(drop = True)
    return new_list_inc['centroid'], u_s_error, u_s

### 2. Assignment of initial centroids

In [ ]:
inc_long_lat = inc[['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad']]
u0 = initiate_centroids(n, inc_long_lat)
u0 = u0.reset_index(drop = True)

In [ ]:
inc_long_lat['centroid'], inc_long_lat['error'] = centroid_assignation(inc_long_lat, u0)

In [ ]:
coords = ['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad']
u1 = inc_long_lat.groupby('centroid').agg('mean').loc[:, coords].reset_index(drop = True)

### 3. Clustering with iterations

In [ ]:
inc_long_lat['centroid'], inc_long_lat['error'], u1 =  clustering(inc_long_lat[coords], n)

### 4. Save and import files (to prevent running code again)

In [ ]:
inc_long_lat.to_csv('incidents_clustered.csv')

In [ ]:
xy_c = np.zeros((n, 2))

for xy in range(n):
    xy_c[xy][0] = u1['primaire_locatie_breedtegraad'][xy]
    xy_c[xy][1] = u1['primaire_locatie_lengtegraad'][xy]

In [ ]:
centroids_coords = pd.DataFrame(xy_c, columns=['breedtegraad', 'lengtegraad'])
centroids_coords.to_csv('centroids_coords.csv')

In [ ]:
transformer1 = Transformer.from_crs("EPSG:4326", "EPSG:28992")
rds_xy = np.zeros((n, 2))

t = transformer1.transform(xy_c['breedtegraad'], xy_c['lengtegraad'])
rds_xy[:, 0] = t[0]
rds_xy[:, 1] = t[1]

In [ ]:
centroids_rds = pd.DataFrame(rds_xy, columns=['breedtegraad', 'lengtegraad'])
centroids_rds.to_csv('centroids_rds.csv')

In [ ]:
new_ill = pd.read_csv('incidents_clustered.csv')
new_cc = pd.read_csv('centroids_coords.csv')
new_rds = pd.read_csv('centroids_rds.csv')

### 5. Plots of clustering based on centroids

#### 5.1. Plot centroids on roadmap

In [ ]:
network_temp.plot(figsize=(12, 10))
plt.plot(new_rds['breedtegraad'], new_rds['lengtegraad'], 'ro');

#### 5.2. Plot incidents and centroids clustered

In [ ]:
plt.figure(figsize=(160, 200))
plt.scatter(new_ill['primaire_locatie_lengtegraad'], new_ill['primaire_locatie_breedtegraad'],  
            marker = 'o', c=new_ill['centroid'].astype('category'), 
            cmap = customcmap, s=150, alpha=0.5)
plt.scatter(new_cc['lengtegraad'], new_cc['breedtegraad'],  
            marker = 's', s=2500, c=np.arange(0, 50, 1), 
            cmap = customcmap)

#### 5.3. Plot cluster sizes

In [ ]:
number_inc_cluster = np.zeros(n)

for i in range(n):
    for j in range(len(new_ill['centroid'])):
        if int(new_ill['centroid'][j]) == i:
            number_inc_cluster[i] += 1
        else:
            number_inc_cluster[i] += 0

In [ ]:
network_temp.plot(figsize=(160, 200), linewidth=5)

plt.scatter(new_rds['breedtegraad'], new_rds['lengtegraad'],  
            marker = 's', s=10*number_inc_cluster, c=np.arange(0, 50, 1), 
            cmap = customcmap)

for t in range(n):
    plt.text(new_rds['breedtegraad'][t] + 1000, new_rds['lengtegraad'][t] + 1000, str(int(number_inc_cluster[t])), size=100)

#### 5.4. Plot average speed of clusters (large approximations)

In [ ]:
v = 80 #km/h
factor = 2.6

In [ ]:
transformer2 = Transformer.from_crs("EPSG:28992", "EPSG:4326")

dist1 = np.zeros((n, 2))
dist2 = np.zeros((len(new_ill['primaire_locatie_breedtegraad']), 2))

t41 = transformer2.transform(new_cc['breedtegraad'], new_cc['lengtegraad'])
t42 = transformer2.transform(new_ill['primaire_locatie_breedtegraad'], new_ill['primaire_locatie_lengtegraad'])

dist1[:, 0] = t41[0]
dist1[:, 1] = t41[1]
dist2[:, 0] = t42[0]
dist2[:, 1] = t42[1]

In [ ]:
tot_time = np.zeros(n)
tot_points = np.zeros(n)
av_time = np.zeros(n)

for i in range(n):
    for j in range(len(new_ill['primaire_locatie_lengtegraad'])):
        if new_ill['centroid'][j] == i:
            absolute_dist = np.sqrt((dist2[:, 1][j] - dist1[:, 1][i])**2 + (dist2[:, 0][j] - dist1[:, 0][i])**2)
            tot_time[i] += absolute_dist * v * factor
            tot_points[i] += 1
        else:
            tot_points[i] += 0
    av_time[i] = tot_time[i] / tot_points[i] * 60 * 1000  # time in minutes

In [ ]:
network_temp.plot(figsize=(160, 200), linewidth=5)

plt.scatter(new_rds['breedtegraad'], new_rds['lengtegraad'],  
            marker = 's', s=10*number_inc_cluster, c=np.arange(0, 50, 1), 
            cmap = customcmap)

for t in range(n):
    plt.text(new_rds['breedtegraad'][t] + 1000, new_rds['lengtegraad'][t] + 1000, f'{av_time[t]:.2f}', size=100)